# Pet Finder System with Image Similarity

This notebook demonstrates a proof-of-concept for a pet finder system that uses image similarity to identify potential matches for lost pets. The system extracts features from pet images focusing on shape and fur/hair characteristics and implements similarity search to find the most similar pets in a database.

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import tensorflow as tf
from tensorflow.keras.applications import ResNet50, VGG16, MobileNetV2, EfficientNetB3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Input, Concatenate
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocess
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg_preprocess
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as mobilenet_preprocess
from tensorflow.keras.applications.efficientnet import preprocess_input as efficientnet_preprocess
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
import seaborn as sns
import requests
from IPython.display import Image, display
import urllib.request
import zipfile
import random
from tqdm import tqdm
import cv2
from skimage.feature import local_binary_pattern
from skimage.color import rgb2hsv
import matplotlib.patches as patches

## 1. Data Preparation

For this demonstration, we'll use the Oxford-IIIT Pet Dataset, which contains images of 37 pet breeds with roughly 200 images for each class. This dataset is perfect for our pet finder system as it contains various breeds with different fur patterns and characteristics.

In [ ]:
# Create data directory
data_dir = "data"
os.makedirs(data_dir, exist_ok=True)

# Download Oxford Pets dataset
def download_oxford_pets():
    # URLs for the dataset
    images_url = "https://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz"
    annotations_url = "https://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz"
    
    # Download and extract images
    print("Downloading and extracting images...")
    images_path = os.path.join(data_dir, "images.tar.gz")
    if not os.path.exists(images_path):
        urllib.request.urlretrieve(images_url, images_path)
        os.system(f"tar -xzf {images_path} -C {data_dir}")
    
    # Download and extract annotations
    print("Downloading and extracting annotations...")
    annotations_path = os.path.join(data_dir, "annotations.tar.gz")
    if not os.path.exists(annotations_path):
        urllib.request.urlretrieve(annotations_url, annotations_path)
        os.system(f"tar -xzf {annotations_path} -C {data_dir}")
    
    print("Dataset downloaded and extracted successfully.")

# Uncomment the line below to download the dataset
# download_oxford_pets()

In [ ]:
# Alternatively, we can use the built-in tensorflow datasets
import tensorflow_datasets as tfds

# Load the dataset
def load_oxford_pets_dataset():
    print("Loading Oxford-IIIT Pet Dataset...")
    dataset, info = tfds.load('oxford_iiit_pet', with_info=True, as_supervised=True)
    return dataset, info

# Uncomment the line below to load the dataset using tensorflow_datasets
# dataset, info = load_oxford_pets_dataset()

## 2. Feature Extractor

Now, let's implement the feature extractor using a pre-trained model. We'll use ResNet50 as our base model, but we'll also implement options for VGG16 and MobileNetV2 to experiment with different feature representations.

In [ ]:
class PetFeatureExtractor:
    def __init__(self, model_type="resnet50"):
        self.model_type = model_type
        self.input_shape = (224, 224, 3)
        self.model, self.preprocess_func = self._build_model()
        
    def _build_model(self):
        """Build a pre-trained model for feature extraction"""
        if self.model_type == "resnet50":
            base_model = ResNet50(weights='imagenet', include_top=False, input_shape=self.input_shape, pooling='avg')
            preprocess = resnet_preprocess
        elif self.model_type == "vgg16":
            base_model = VGG16(weights='imagenet', include_top=False, input_shape=self.input_shape, pooling='avg')
            preprocess = vgg_preprocess
        elif self.model_type == "mobilenetv2":
            base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=self.input_shape, pooling='avg')
            preprocess = mobilenet_preprocess
        elif self.model_type == "efficientnet":
            base_model = EfficientNetB3(weights='imagenet', include_top=False, input_shape=self.input_shape, pooling='avg')
            preprocess = efficientnet_preprocess
        else:
            raise ValueError(f"Model type {self.model_type} not supported")
            
        return base_model, preprocess
    
    def extract_features(self, img_path):
        """Extract features from a single image"""
        # Load and preprocess the image
        img = image.load_img(img_path, target_size=(self.input_shape[0], self.input_shape[1]))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = self.preprocess_func(img_array)
        
        # Extract features
        features = self.model.predict(img_array, verbose=0)
        return features[0]  # Return the feature vector
    
    def extract_features_batch(self, img_paths, batch_size=32):
        """Extract features from a batch of images"""
        features = []
        
        # Process images in batches to avoid memory issues
        for i in tqdm(range(0, len(img_paths), batch_size)):
            batch_paths = img_paths[i:i+batch_size]
            batch_size = len(batch_paths)
            img_arrays = np.zeros((batch_size, self.input_shape[0], self.input_shape[1], 3))
            
            for j, img_path in enumerate(batch_paths):
                try:
                    img = image.load_img(img_path, target_size=(self.input_shape[0], self.input_shape[1]))
                    img_arrays[j] = image.img_to_array(img)
                except Exception as e:
                    print(f"Error processing {img_path}: {str(e)}")
                    # Use zeros for failed images
                    img_arrays[j] = np.zeros(self.input_shape)
            
            img_arrays = self.preprocess_func(img_arrays)
            batch_features = self.model.predict(img_arrays, verbose=0)
            features.append(batch_features)
        
        return np.vstack(features)
    
    def extract_features_from_array(self, img_array):
        """Extract features from a preprocessed image array"""
        img_array = np.expand_dims(img_array, axis=0)
        img_array = self.preprocess_func(img_array)
        features = self.model.predict(img_array, verbose=0)
        return features[0]

## 3. Pet Database

Next, let's implement a simple in-memory database to store pet information and features. In a real-world application, this would be a full database, but for our PoC, we'll use pandas DataFrames.

In [ ]:
class PetDatabase:
    def __init__(self):
        """Initialize the pet database"""
        # Create a DataFrame to store pet metadata
        self.pets_df = pd.DataFrame(columns=[
            'id', 'name', 'species', 'breed', 'color', 
            'location', 'date_added', 'status', 'image_path'
        ])
        
        # Store features separately as they're large
        self.features = []
        self.next_id = 1
    
    def add_pet(self, metadata, features):
        """Add a pet to the database with its metadata and features"""
        # Assign an ID and add to metadata
        pet_id = self.next_id
        self.next_id += 1
        
        # Add current date if not provided
        if 'date_added' not in metadata:
            metadata['date_added'] = datetime.now().strftime('%Y-%m-%d')
        
        # Add to DataFrame
        pet_data = {'id': pet_id, **metadata}
        self.pets_df = pd.concat([self.pets_df, pd.DataFrame([pet_data])], ignore_index=True)
        
        # Store features
        self.features.append(features)
        
        return pet_id
    
    def search_similar_pets(self, query_features, top_n=5, status=None, location=None, max_days=None):
        """Find pets with similar features to the query"""
        if len(self.features) == 0:
            return []
        
        # Convert features to numpy array
        features_array = np.array(self.features)
        
        # Calculate cosine similarity
        similarities = cosine_similarity([query_features], features_array)[0]
        
        # Create a copy of the DataFrame to filter
        filtered_df = self.pets_df.copy()
        
        # Apply filters
        if status:
            filtered_df = filtered_df[filtered_df['status'] == status]
        
        if location:
            filtered_df = filtered_df[filtered_df['location'] == location]
        
        if max_days:
            # Calculate days since added
            today = datetime.now().date()
            filtered_df['days_since'] = filtered_df['date_added'].apply(
                lambda x: (today - datetime.strptime(x, '%Y-%m-%d').date()).days
            )
            filtered_df = filtered_df[filtered_df['days_since'] <= max_days]
        
        # Get indices that remain after filtering
        valid_indices = filtered_df.index.tolist()
        
        # Add similarity to the DataFrame only for valid indices
        results_df = filtered_df.copy()
        results_df['similarity'] = np.nan
        
        for i in valid_indices:
            if i < len(similarities):
                results_df.loc[i, 'similarity'] = similarities[i]
        
        # Sort by similarity and get top N
        results_df = results_df.dropna(subset=['similarity'])
        results_df = results_df.sort_values('similarity', ascending=False).head(top_n)
        
        # Convert to list of dictionaries
        results = results_df.to_dict('records')
        return results
    
    def get_pet_by_id(self, pet_id):
        """Retrieve a pet by its ID"""
        pet = self.pets_df[self.pets_df['id'] == pet_id]
        if len(pet) == 0:
            return None
        return pet.iloc[0].to_dict()
    
    def update_pet_status(self, pet_id, new_status):
        """Update the status of a pet (lost, found, reunited)"""
        idx = self.pets_df[self.pets_df['id'] == pet_id].index
        if len(idx) > 0:
            self.pets_df.loc[idx[0], 'status'] = new_status
            return True
        return False
    
    def get_features_array(self):
        """Get all features as a numpy array"""
        return np.array(self.features)
    
    def get_all_pets(self):
        """Get all pets in the database"""
        return self.pets_df

## 4. Demo with Sample Data

For demonstration purposes, we'll create a small dataset from the Oxford-IIIT Pet Dataset. We'll extract features from a subset of images and simulate a lost pet scenario.

In [ ]:
# Check if the images directory exists, otherwise use a small demo dataset
def get_image_paths(directory=None, limit=100):
    """Get image paths from directory or use demo images"""
    if directory and os.path.exists(directory):
        image_paths = []
        for root, _, files in os.walk(directory):
            for file in files:
                if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                    image_paths.append(os.path.join(root, file))
        
        # Limit the number of images for demo
        if limit and len(image_paths) > limit:
            return random.sample(image_paths, limit)
        return image_paths
    else:
        # Use demo images or download a small sample
        print("Using demo images...")
        demo_dir = os.path.join(data_dir, "demo_images")
        os.makedirs(demo_dir, exist_ok=True)
        
        # Add code here to download some sample images
        # For simplicity, we'll just return an empty list for now
        return []

# Function to extract breed from filename
def extract_metadata_from_filename(filename):
    """Extract breed and species from Oxford-IIIT Pet Dataset filename"""
    # Example: Abyssinian_1.jpg
    parts = os.path.basename(filename).split('_')
    breed = parts[0].replace('_', ' ')
    
    # Determine species (cat or dog)
    cat_breeds = ['Abyssinian', 'Bengal', 'Birman', 'Bombay', 'British', 'Egyptian', 'Maine', 'Persian', 'Ragdoll', 'Russian', 'Siamese', 'Sphynx']
    species = 'cat' if any(cat in breed for cat in cat_breeds) else 'dog'
    
    return {
        'name': f"Pet_{os.path.basename(filename).split('.')[0]}",
        'species': species,
        'breed': breed,
        'status': 'lost' if random.random() > 0.5 else 'found',
        'location': random.choice(['Downtown', 'Uptown', 'Midtown', 'Suburb', 'Park']),
        'date_added': (datetime.now() - pd.Timedelta(days=random.randint(0, 30))).strftime('%Y-%m-%d'),
        'image_path': filename
    }

In [ ]:
# Initialize our feature extractor and database
feature_extractor = PetFeatureExtractor(model_type="resnet50")
pet_db = PetDatabase()

# Load a subset of images
image_dir = os.path.join(data_dir, "images")
image_paths = get_image_paths(image_dir, limit=200)

# If we have images, process them
if image_paths:
    print(f"Processing {len(image_paths)} images...")
    
    # Extract features for all images
    features = feature_extractor.extract_features_batch(image_paths)
    
    # Add each pet to the database
    for i, img_path in enumerate(image_paths):
        metadata = extract_metadata_from_filename(img_path)
        pet_db.add_pet(metadata, features[i])
    
    print(f"Added {len(image_paths)} pets to the database")
else:
    print("No images found. Please download the dataset or provide sample images.")

## 5. Visualize Feature Space

Let's visualize the feature space to better understand how our model separates different breeds and species.

In [ ]:
def visualize_feature_space(db, n_samples=100):
    """Visualize the feature space using t-SNE"""
    if len(db.features) == 0:
        print("No features to visualize")
        return
    
    # Get features and metadata
    features = db.get_features_array()
    pets_df = db.get_all_pets()
    
    # If we have too many samples, subsample
    if len(features) > n_samples:
        indices = np.random.choice(len(features), n_samples, replace=False)
        features = features[indices]
        pets_df = pets_df.iloc[indices].reset_index(drop=True)
    
    # Apply t-SNE for dimensionality reduction
    print("Applying t-SNE dimensionality reduction...")
    tsne = TSNE(n_components=2, random_state=42)
    features_2d = tsne.fit_transform(features)
    
    # Create a DataFrame with t-SNE features
    tsne_df = pd.DataFrame({
        'x': features_2d[:, 0],
        'y': features_2d[:, 1],
        'breed': pets_df['breed'],
        'species': pets_df['species'],
        'status': pets_df['status']
    })
    
    # Plot by species
    plt.figure(figsize=(12, 10))
    sns.scatterplot(data=tsne_df, x='x', y='y', hue='species', style='status', s=100, alpha=0.7)
    plt.title('t-SNE Visualization of Pet Features by Species')
    plt.legend(title='Species', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()
    
    # Plot by breed (only show top 10 breeds for clarity)
    top_breeds = pets_df['breed'].value_counts().head(10).index.tolist()
    breed_tsne_df = tsne_df[tsne_df['breed'].isin(top_breeds)]
    
    plt.figure(figsize=(14, 12))
    sns.scatterplot(data=breed_tsne_df, x='x', y='y', hue='breed', s=100, alpha=0.7)
    plt.title('t-SNE Visualization of Pet Features by Breed (Top 10)')
    plt.legend(title='Breed', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()

# Uncomment to visualize the feature space
visualize_feature_space(pet_db)

## 6. Simulating a Lost Pet Search

Now, let's simulate a search for a lost pet. We'll pick a random pet from our database, pretend it's a lost pet, and search for similar pets.

In [ ]:
def simulate_pet_search(db, feature_extractor, num_pets=5):
    """Simulate searching for lost pets"""
    if len(db.pets_df) == 0:
        print("No pets in the database to search for.")
        return
    
    # Select random pets to search for
    sample_pets = db.pets_df.sample(num_pets)
    
    for _, pet in sample_pets.iterrows():
        print(f"\n{'='*50}")
        print(f"Searching for similar pets to: {pet['name']} ({pet['breed']}, {pet['species']})")
        print(f"Status: {pet['status']}, Location: {pet['location']}")
        
        # Display the pet image
        plt.figure(figsize=(6, 6))
        img = image.load_img(pet['image_path'])
        plt.imshow(np.array(img))
        plt.title(f"Query: {pet['breed']}")
        plt.axis('off')
        plt.show()
        
        # Get the index of this pet in our features list
        pet_idx = db.pets_df[db.pets_df['id'] == pet['id']].index[0]
        query_features = db.features[pet_idx]
        
        # Search for similar pets
        results = db.search_similar_pets(query_features, top_n=5)
        
        # Filter out the query pet itself
        results = [r for r in results if r['id'] != pet['id']]
        
        if not results:
            print("No similar pets found.")
            continue
        
        # Display results
        print(f"\nFound {len(results)} similar pets:")
        
        plt.figure(figsize=(15, 4))
        for i, result in enumerate(results[:4]):
            plt.subplot(1, 4, i+1)
            img = image.load_img(result['image_path'])
            plt.imshow(np.array(img))
            plt.title(f"Match {i+1}: {result['breed']}\nScore: {result['similarity']:.2f}")
            plt.axis('off')
            print(f"{i+1}. {result['breed']} ({result['species']})")
            print(f"   Similarity: {result['similarity']:.2f}")
            print(f"   Status: {result['status']}, Location: {result['location']}")
        
        plt.tight_layout()
        plt.show()

# Uncomment to simulate a pet search
simulate_pet_search(pet_db, feature_extractor, num_pets=3)

## 7. Improving Feature Extraction

The basic ResNet50 features might not be specific enough for pet identification. Let's implement some additional techniques to improve our feature extraction:

In [ ]:
class EnhancedPetFeatureExtractor(PetFeatureExtractor):
    def __init__(self, model_type="resnet50", use_color_histogram=True, use_texture_features=True):
        super().__init__(model_type)
        self.use_color_histogram = use_color_histogram
        self.use_texture_features = use_texture_features
    
    def compute_color_histogram(self, img_array):
        """Compute color histogram features with HSV color space"""
        # Convert to HSV color space (better for color analysis)
        hsv_img = rgb2hsv(img_array / 255.0)
        
        # Extract histograms for each channel
        h_hist = np.histogram(hsv_img[:,:,0], bins=32, range=(0, 1))[0]
        s_hist = np.histogram(hsv_img[:,:,1], bins=32, range=(0, 1))[0]
        v_hist = np.histogram(hsv_img[:,:,2], bins=32, range=(0, 1))[0]
        
        # Normalize histograms
        h_hist = h_hist / np.sum(h_hist) if np.sum(h_hist) > 0 else h_hist
        s_hist = s_hist / np.sum(s_hist) if np.sum(s_hist) > 0 else s_hist
        v_hist = v_hist / np.sum(v_hist) if np.sum(v_hist) > 0 else v_hist
        
        # Add statistical features (mean and std) for each channel
        h_mean, h_std = np.mean(hsv_img[:,:,0]), np.std(hsv_img[:,:,0])
        s_mean, s_std = np.mean(hsv_img[:,:,1]), np.std(hsv_img[:,:,1])
        v_mean, v_std = np.mean(hsv_img[:,:,2]), np.std(hsv_img[:,:,2])
        
        # Combine features
        color_features = np.concatenate([
            h_hist, s_hist, v_hist,
            [h_mean, h_std, s_mean, s_std, v_mean, v_std]
        ])
        
        return color_features
    
    def compute_texture_features(self, img_array):
        """Compute texture features using Local Binary Patterns"""
        # Convert to grayscale
        gray = cv2.cvtColor(img_array.astype(np.uint8), cv2.COLOR_RGB2GRAY)
        
        # Parameters for LBP
        radius = 3
        n_points = 8 * radius
        
        # Compute LBP
        lbp = local_binary_pattern(gray, n_points, radius, method='uniform')
        
        # Compute histogram of LBP
        n_bins = n_points + 2  # uniform LBP has n_points + 2 distinct values
        lbp_hist, _ = np.histogram(lbp, bins=n_bins, range=(0, n_bins))
        
        # Normalize histogram
        lbp_hist = lbp_hist / np.sum(lbp_hist) if np.sum(lbp_hist) > 0 else lbp_hist
        
        return lbp_hist
    
    def extract_features(self, img_path):
        """Extract enhanced features from a single image"""
        # Load and preprocess the image
        img = image.load_img(img_path, target_size=(self.input_shape[0], self.input_shape[1]))
        img_array = image.img_to_array(img)
        
        # Extract deep features
        deep_features = super().extract_features(img_path)
        
        features_list = [deep_features]
        
        # Add color histogram features
        if self.use_color_histogram:
            color_features = self.compute_color_histogram(img_array)
            features_list.append(color_features)
        
        # Add texture features
        if self.use_texture_features:
            texture_features = self.compute_texture_features(img_array)
            features_list.append(texture_features)
        
        # Concatenate all features
        combined_features = np.concatenate(features_list)
        return combined_features
    
    def extract_features_batch(self, img_paths, batch_size=16):
        """Extract enhanced features from a batch of images"""
        # Process in smaller batches due to additional feature extraction
        features = []
        for img_path in tqdm(img_paths):
            try:
                features.append(self.extract_features(img_path))
            except Exception as e:
                print(f"Error processing {img_path}: {str(e)}")
                # Use zeros with the right dimension
                if len(features) > 0:
                    features.append(np.zeros_like(features[0]))
                else:
                    # Make an educated guess about the feature size
                    deep_feature_size = 2048  # for ResNet50
                    color_hist_size = 102  # 32 bins * 3 channels + 6 statistics
                    texture_hist_size = 26  # uniform LBP with radius 3
                    total_size = deep_feature_size
                    if self.use_color_histogram:
                        total_size += color_hist_size
                    if self.use_texture_features:
                        total_size += texture_hist_size
                    features.append(np.zeros(total_size))
        
        return np.array(features)

## 8. Demo with Enhanced Features

Let's try our enhanced feature extractor and compare the results.

In [ ]:
# Initialize our enhanced feature extractor and a new database
enhanced_extractor = EnhancedPetFeatureExtractor(model_type="resnet50")
enhanced_db = PetDatabase()

# If we have images, process them with enhanced features
if image_paths:
    print(f"Processing {len(image_paths)} images with enhanced features...")
    
    # Extract enhanced features for all images
    enhanced_features = enhanced_extractor.extract_features_batch(image_paths)
    
    # Add each pet to the database with enhanced features
    for i, img_path in enumerate(image_paths):
        metadata = extract_metadata_from_filename(img_path)
        enhanced_db.add_pet(metadata, enhanced_features[i])
    
    print(f"Added {len(image_paths)} pets to the enhanced database")
    
    # Simulate a pet search with enhanced features
    # Uncomment to run the simulation
    simulate_pet_search(enhanced_db, enhanced_extractor, num_pets=3)

## 9. Adding Metadata to Improve Matching

Now, let's incorporate metadata like location to improve our search results. We'll implement a weighted search that combines feature similarity with metadata matching.

In [ ]:
class MetadataEnhancedPetDatabase(PetDatabase):
    def search_similar_pets(self, query_features, query_metadata=None, top_n=5, feature_weight=0.7):
        """Find pets with similar features and metadata"""
        if len(self.features) == 0:
            return []
        
        # Calculate feature similarity
        features_array = np.array(self.features)
        feature_similarities = cosine_similarity([query_features], features_array)[0]
        
        # Create a DataFrame with results
        results_df = self.pets_df.copy()
        results_df['feature_similarity'] = feature_similarities
        
        # Calculate metadata similarity if provided
        if query_metadata:
            # Initialize metadata similarity score
            results_df['metadata_similarity'] = 0.0
            
            # Check location match
            if 'location' in query_metadata and query_metadata['location']:
                results_df.loc[results_df['location'] == query_metadata['location'], 'metadata_similarity'] += 0.5
            
            # Check species match
            if 'species' in query_metadata and query_metadata['species']:
                results_df.loc[results_df['species'] == query_metadata['species'], 'metadata_similarity'] += 0.3
            
            # Check breed match
            if 'breed' in query_metadata and query_metadata['breed']:
                results_df.loc[results_df['breed'] == query_metadata['breed'], 'metadata_similarity'] += 0.2
            
            # Combine feature and metadata similarity
            results_df['similarity'] = (
                feature_weight * results_df['feature_similarity'] + 
                (1 - feature_weight) * results_df['metadata_similarity']
            )
        else:
            # Use only feature similarity
            results_df['similarity'] = results_df['feature_similarity']
        
        # Sort by combined similarity and get top N
        results_df = results_df.sort_values('similarity', ascending=False).head(top_n)
        
        # Convert to list of dictionaries
        results = results_df.to_dict('records')
        return results

In [ ]:
# Initialize our metadata-enhanced database
metadata_db = MetadataEnhancedPetDatabase()

# If we have images, process them
if image_paths:
    print(f"Processing {len(image_paths)} images for the metadata-enhanced database...")
    
    # Reuse the enhanced features
    for i, img_path in enumerate(image_paths):
        metadata = extract_metadata_from_filename(img_path)
        metadata_db.add_pet(metadata, enhanced_features[i])
    
    print(f"Added {len(image_paths)} pets to the metadata-enhanced database")
    
    # Define a function to simulate search with metadata
    def simulate_metadata_search(db, extractor, num_pets=3):
        """Simulate search with metadata enhancement"""
        if len(db.pets_df) == 0:
            print("No pets in the database to search for.")
            return
        
        # Select random lost pets to search for
        lost_pets = db.pets_df[db.pets_df['status'] == 'lost'].sample(num_pets)
        
        for _, pet in lost_pets.iterrows():
            print(f"\n{'='*50}")
            print(f"Searching for similar pets to: {pet['name']} ({pet['breed']}, {pet['species']})")
            print(f"Location: {pet['location']}")
            
            # Display the pet image
            plt.figure(figsize=(6, 6))
            img = image.load_img(pet['image_path'])
            plt.imshow(np.array(img))
            plt.title(f"Lost pet: {pet['breed']}")
            plt.axis('off')
            plt.show()
            
            # Get the index of this pet in our features list
            pet_idx = db.pets_df[db.pets_df['id'] == pet['id']].index[0]
            query_features = db.features[pet_idx]
            
            # Create query metadata
            query_metadata = {
                'species': pet['species'],
                'location': pet['location']
            }
            
            # Search with feature similarity only
            feature_results = db.search_similar_pets(query_features, top_n=3)
            # Filter out the query pet itself
            feature_results = [r for r in feature_results if r['id'] != pet['id']]
            
            # Search with metadata enhancement
            metadata_results = db.search_similar_pets(query_features, query_metadata, top_n=3)
            # Filter out the query pet itself
            metadata_results = [r for r in metadata_results if r['id'] != pet['id']]
            
            # Display feature-only results
            if feature_results:
                print("\nResults using only image features:")
                plt.figure(figsize=(15, 4))
                for i, result in enumerate(feature_results[:3]):
                    plt.subplot(1, 3, i+1)
                    img = image.load_img(result['image_path'])
                    plt.imshow(np.array(img))
                    plt.title(f"Match {i+1}: {result['breed']}\nScore: {result['similarity']:.2f}")
                    plt.axis('off')
                    print(f"{i+1}. {result['breed']} ({result['species']})")
                    print(f"   Similarity: {result['similarity']:.2f}")
                    print(f"   Location: {result['location']}")
                plt.tight_layout()
                plt.show()
            
            # Display metadata-enhanced results
            if metadata_results:
                print("\nResults using image features + metadata:")
                plt.figure(figsize=(15, 4))
                for i, result in enumerate(metadata_results[:3]):
                    plt.subplot(1, 3, i+1)
                    img = image.load_img(result['image_path'])
                    plt.imshow(np.array(img))
                    plt.title(f"Match {i+1}: {result['breed']}\nScore: {result['similarity']:.2f}")
                    plt.axis('off')
                    print(f"{i+1}. {result['breed']} ({result['species']})")
                    print(f"   Similarity: {result['similarity']:.2f}")
                    print(f"   Location: {result['location']}")
                plt.tight_layout()
                plt.show()
    
    # Uncomment to run the metadata-enhanced search
    simulate_metadata_search(metadata_db, enhanced_extractor, num_pets=3)